In [1]:
import torch
import torch.nn as nn
from torch import optim
import matplotlib.pyplot as plt
import torchvision
from torchvision import transforms
import torch.nn.functional as F
import numpy as np

In [2]:
torch.__version__

'2.0.1+cpu'

# 数据准备

In [3]:
# 数据归一化，归一化到-1到1
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(0.5,0.5)
])

In [4]:
train_ds = torchvision.datasets.MNIST('data',train=True,transform=transform,download=True)

In [5]:
dataloader = torch.utils.data.DataLoader(train_ds,batch_size=64,shuffle=True)

In [6]:
imgs,_ = next(iter(dataloader))

In [7]:
# plt.imshow(imgs[2,0].numpy())

In [8]:
imgs.shape

torch.Size([64, 1, 28, 28])

# 定义生成器

In [9]:
class Generator(nn.Module):
    def __init__(self):
        super(Generator,self).__init__()
        self.main = nn.Sequential(
            nn.Linear(100,256),
            nn.ReLU(),
            nn.Linear(256,512),
            nn.ReLU(),
            nn.Linear(512,28*28),
            nn.Tanh()
        )

    def forward(self,x):
        img = self.main(x)
        return img.view(-1,28,28,1)

# 定义判别器

In [10]:
class Discrimeinator(nn.Module):
    def __init__(self):
        super(Discrimeinator,self).__init__()
        self.main = nn.Sequential(
            nn.Linear(28*28,512),
            nn.LeakyReLU(),
            nn.Linear(512,256),
            nn.LeakyReLU(),
            nn.Linear(256,1),
            nn.Sigmoid()
        )
    def forward(self,x):
        x = x.view(-1,28*28)
        x = self.main(x)
        return x

# 初始化模型，优化器及损失函数计算

In [11]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [12]:
gen = Generator().to(device=device)
dis = Discrimeinator().to(device=device)

In [13]:
g_optim = torch.optim.Adam(gen.parameters(),lr=0.001)
d_optim = torch.optim.Adam(dis.parameters(),lr=0.001)

In [14]:
loss_fn = nn.BCELoss()

# 绘图函数

In [15]:
def gen_img_plot(model,epoch,test_input):
    pred = np.squeeze(model(test_input).detach().cpu().numpy())
    fig = plt.figure(figsize=(4,4))
    for i in range(16):
        plt.subplot(4,4,i+1)
        plt.imshow((pred[i]+1)/2)
        plt.axis('off')
    plt.show()

In [16]:
test_input = torch.randn(16,100,device=device)

# GAN的训练

In [17]:
D_loss = []
G_loss = []

In [22]:
epoch = 20

In [25]:
for epoch in range(epoch):
    d_epoch_loss = 0
    g_epoch_loss = 0
    count = len(dataloader)
    for step,(img,_) in enumerate(dataloader):
        img = img.to(device)
        size = img.size(0)
        random_noise = torch.randn(size,100).to(device=device)

        # 真实数据经过鉴别器并更新鉴别器
        d_optim.zero_grad()
        real_output = dis(img)
        d_real_loss = loss_fn(real_output,torch.ones_like(real_output))
        d_real_loss.backward()

        # 造假数据经过鉴别器并更新鉴别器
        gen_img = gen(random_noise)
        fake_output = dis(gen_img.detach())
        d_fake_loss = loss_fn(fake_output,torch.zeros_like(fake_output))
        d_fake_loss.backward()
        d_loss = d_real_loss + d_fake_loss
        d_optim.step()
        # 造假数据经过鉴别器并更新生成器
        g_optim.zero_grad()
        fake_output = dis(gen_img)
        g_loss = loss_fn(fake_output,torch.ones_like(fake_output))
        g_loss.backward()
        g_optim.step()

        with torch.no_grad():
            d_epoch_loss += d_loss
            g_epoch_loss += g_loss
    with torch.no_grad():
        d_epoch_loss /= count
        g_epoch_loss /= count
        D_loss.append(d_epoch_loss)
        G_loss.append(g_epoch_loss)
        # if step%100 == 0:
        print('Epoch{}nsteps:{}'.format(epoch,step))
        gen_img_plot(gen,epoch,test_input)